# Sudoku Solver

In [ ]:
# Necessary Imports

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random
import cv2
from glob import glob

# Step 1 : Building Digit Classification Model

## Loading the Dataset

In [ ]:
path = r"../Digits_Dataset/digits updated"
data = os.listdir(path)

data_X = []     
data_y = [] 

data_classes = len(data)
# print(data_classes)

for i in range (0,data_classes):
    data_list = os.listdir(path +"/"+str(i))
    # print(len(data_list))
    for j in data_list:
        pic = cv2.imread(path +"/"+str(i)+"/"+j)
        pic = cv2.resize(pic,(32,32))
        data_X.append(pic)
        data_y.append(i)
# plt.imshow(data_X[-5040])
# print(data_y[-5040])


if len(data_X) == len(data_y) :
    print("Total Dataponits = ",len(data_X))

# Labels and images
data_X = np.array(data_X)
data_y = np.array(data_y)

In [ ]:
import random

from sklearn.utils import shuffle
data_X, data_y = shuffle(data_X, data_y)

# temp_list = data_X;
# random.shuffle(temp_list)

plt.figure(figsize=(10, 10))
for i in range(8000, 8025):
    plt.subplot(5, 5, i+1-8000)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_X[i])
    plt.xlabel(str(data_y[i]))
plt.show()

## Splitting the Dataset

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(data_X,data_y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)

print("Training Set Shape = ",train_X.shape, " ; Percentage : ", round(train_X.shape[0]/len(data_X) * 100, 2),"%")
print("Validation Set Shape = ",valid_X.shape, " ; Percentage : ", round(valid_X.shape[0]/len(data_X) * 100, 2),"%")
print("Test Set Shape = ",test_X.shape, " ; Percentage : ", round(test_X.shape[0]/len(data_X) * 100, 2),"%")

## Preprocessing the Image

In [ ]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator

# Preprocessing the images for neuralnet

def Prep(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #making image grayscale
    img = cv2.equalizeHist(img) #Histogram equalization to enhance contrast
    img = img/255 #normalizing
    return img

# map function : map() function returns a map object(which is an iterator) of the results after applying the given function to each item of a given iterable (list, tuple etc.)

train_X = np.array(list(map(Prep, train_X)))
test_X = np.array(list(map(Prep, test_X)))
valid_X= np.array(list(map(Prep, valid_X)))



# Reshaping the images
# The reshaping step adjusts the dimensions of the images to include a channel dimension (necessary for neural networks):

train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)

# Augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(train_X)

In [ ]:
from keras.src.utils import to_categorical

# One hot encoding of the labels

train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

## Model Building

### Normal Model made from scratch

In [ ]:
from keras.src.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras import Sequential

#Creating a Neural Network

model = Sequential()

model.add((Conv2D(60,(5,5),input_shape=(32, 32, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

#Compiling the model

optimizer = RMSprop(learning_rate =0.001, rho=0.9, epsilon = 1e-08, decay=0.0)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#Fit the model

history = model.fit(datagen.flow(train_X, train_y, batch_size=32),
                              epochs = 30, validation_data = (valid_X, valid_y),
                              verbose = 2, steps_per_epoch= 200)

In [ ]:
# Testing the model on the test set

score = model.evaluate(test_X, test_y, verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

In [ ]:
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
model.save('model_1.h5')

In [14]:
img = cv2.imread('C:/Users/Mayank/OneDrive/Pictures/Screenshots/Screenshot 2024-06-28 231224.png')
img = cv2.resize(img,(32,32))

img = Prep(img)
img = np.array(img)

img = img.reshape(1, 32, 32, 1)
pred = model.predict(img)
clas = pred.argmax()

print(clas)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
6
